In [ ]:
!pip install --upgrade "qiskit[all]" qiskit-nature qiskit-aer
!pip install "qiskit<2" 
!pip install qiskit-optimization
!pip install qiskit-finance


In [1]:
import numpy as np

from qiskit_finance.applications import PortfolioOptimization
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver, SamplingVQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_aer.primitives import Sampler

# ----- Your data -----
mu = np.array([0.10, 0.20, 0.15, 0.12])
sigma = np.array([
    [ 0.005, -0.010,  0.004, -0.002],
    [-0.010,  0.040, -0.002,  0.004],
    [ 0.004, -0.002,  0.023,  0.002],
    [-0.002,  0.004,  0.002,  0.015]
])
q = 0.5        # risk factor (λ)
budget = 2     # select exactly 2 assets

# ----- Build QuadraticProgram from the Finance app -----
portfolio = PortfolioOptimization(
    expected_returns=mu,
    covariances=sigma,
    risk_factor=q,
    budget=budget
)
qp = portfolio.to_quadratic_program()   # penalty handled internally

def show(result, title):
    x = result.x.astype(int)
    print(f"\n== {title} ==")
    print("Selection (1=invest):", x, "  cost:", result.fval)
    print("Picked asset indices:", [i for i, v in enumerate(x) if v == 1])
    # optional: report portfolio stats
    print("Expected return:", float(mu @ x))
    print("Variance:", float(x @ sigma @ x))

# ----- Classical reference (exact on CPU) -----
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())
res_exact = exact.solve(qp)
show(res_exact, "Classical Exact (NumPyMinimumEigensolver)")

# ----- QAOA (Sampler primitive) -----
sampler = Sampler()
qaoa = QAOA(sampler=sampler, optimizer=COBYLA(maxiter=250), reps=3)
qaoa_solver = MinimumEigenOptimizer(qaoa)
res_qaoa = qaoa_solver.solve(qp)
show(res_qaoa, "QAOA")



== Classical Exact (NumPyMinimumEigensolver) ==
Selection (1=invest): [0 1 1 0]   cost: -0.3205
Picked asset indices: [1, 2]
Expected return: 0.35
Variance: 0.059

== QAOA ==
Selection (1=invest): [0 1 1 0]   cost: -0.3205
Picked asset indices: [1, 2]
Expected return: 0.35
Variance: 0.059
